In [14]:
import pickle
import pandas as pd
import numpy as np
import os
d = "raw/pk/"
files = sorted([d + f for f in os.listdir(d) if f.endswith("lpow")])

In [15]:
def getdf(loc):
    df = pd.read_csv(loc, comment="#", delim_whitespace=True, names=["i", "k", "pk", "nk"])
    mask = (df["nk"] > 0) & (df["k"] <= 0.4)
    masked = df.loc[mask, ["k", "pk", "nk"]]
    return masked.astype(np.float32)

def get_winfits(directory):
    files = sorted([directory + f for f in os.listdir(directory) if "winfit" in f])
    print(files)
    winfits = {}
    for winfit_file in files:
        step_size = int(winfit_file.split(".")[-2].split("_")[-1])
        print(f"Loading winfit from {winfit_file}")
        matrix = np.genfromtxt(winfit_file, skip_header=4)
        res = {}
        winfits[step_size] = res
        res["w_ks_input"] = matrix[:, 0]
        res["w_k0_scale"] = matrix[:, 1]
        res["w_transform"] = matrix[:, 2:]/(np.sum(matrix[:, 2:], axis=0))
        # God I am sorry for doing this manually but the file format is... tricky
        with open(winfit_file, "r") as f:
            res["w_ks_output"] = np.array([float(x) for x in f.readlines()[2].split()[1:]])
    return winfits

def get_winpk(directory):
    files = sorted([directory + f for f in os.listdir(directory) if "lwin" in f])
    assert len(files) == 1, str(files)
    print(files[0])
    data = np.genfromtxt(files[0])
    return data

res = {f.lower(): getdf(f) for f in files}
split = {
    "pre-recon": [v for k, v in res.items() if "recon" not in k],
    "post-recon": [v for k, v in res.items() if "recon" in k],
    "cosmology": {"om": 0.31, "h0": 0.67, "z": 0.15, "ob": 0.0491, "ns": 0.96, "reconsmoothscale": 21.21},
    "name": "SDSS MGS DR7 Pk",
    "winfit": get_winfits("./"),
    "winpk": get_winpk("./")
}

['./bin0_winfit_10.txt', './bin0_winfit_5.txt', './bin0_winfit_7.txt']
Loading winfit from ./bin0_winfit_10.txt
Loading winfit from ./bin0_winfit_5.txt
Loading winfit from ./bin0_winfit_7.txt
./lwin.txt


In [16]:
with open("../sdss_dr7_pk.pkl", "wb") as f:
    pickle.dump(split, f)